In [1]:
import requests

In [2]:
standings_url = "https://fbref.com/en/comps/13/Ligue-1-Stats"

In [3]:
data = requests.get(standings_url)

In [4]:
from bs4 import BeautifulSoup

In [5]:
soup = BeautifulSoup(data.text)

In [6]:
standings_table = soup.select('table.stats_table')[0]

In [7]:
soup

<!DOCTYPE html>
<html class="no-js" data-root="/home/fb/deploy/www/base" data-version="klecko-" lang="en">
<head>
<meta charset="utf-8"/>
<meta content="ie=edge" http-equiv="x-ua-compatible"/>
<meta content="width=device-width, initial-scale=1.0, maximum-scale=2.0" name="viewport"/>
<link href="https://cdn.ssref.net/req/202307011" rel="dns-prefetch"/>
<!-- Quantcast Choice. Consent Manager Tag v2.0 (for TCF 2.0) -->
<script async="true" type="text/javascript">
    (function() {
	var host = window.location.hostname;
	var element = document.createElement('script');
	var firstScript = document.getElementsByTagName('script')[0];
	var url = 'https://cmp.quantcast.com'
	    .concat('/choice/', 'XwNYEpNeFfhfr', '/', host, 
		    '/choice.js?tag_version=V2');
	var uspTries = 0;
	var uspTriesLimit = 3;
	element.async = true;
	element.type = 'text/javascript';
	element.src = url;
	
	firstScript.parentNode.insertBefore(element, firstScript);
	
	function makeStub() {
	    var TCF_LOCATOR_NAME = '_

In [8]:
links = standings_table.find_all('a')

In [9]:
links = [l.get("href") for l in links]

In [10]:
links = [l for l in links if '/squads/' in l]

In [11]:
team_urls = [f"https://fbref.com{l}" for l in links]

In [12]:
data = requests.get(team_urls[0])

In [13]:
import pandas as pd
matches = pd.read_html(data.text, match="Scores & Fixtures")[0]

In [14]:
soup = BeautifulSoup(data.text)
links = soup.find_all('a')
links = [l.get("href") for l in links]
links = [l for l in links if l and 'all_comps/shooting/' in l]

In [15]:
data = requests.get(f"https://fbref.com{links[0]}")

In [16]:
shooting = pd.read_html(data.text, match="Shooting")[0]

In [17]:
shooting.head()

For Paris Saint-Germain                                \
                     Date   Time                   Comp   
0              2022-07-31  21:00  Trophée des Champions   
1              2022-08-06  21:00                Ligue 1   
2              2022-08-13  21:00                Ligue 1   
3              2022-08-21  20:45                Ligue 1   
4              2022-08-28  20:45                Ligue 1   

                                                                 ... Standard  \
                   Round  Day Venue Result GF GA       Opponent  ...     Dist   
0  Trophée des Champions  Sun  Home      W  4  0         Nantes  ...      NaN   
1            Matchweek 1  Sat  Away      W  5  0  Clermont Foot  ...     12.9   
2            Matchweek 2  Sat  Home      W  5  2    Montpellier  ...     18.2   
3            Matchweek 3  Sun  Away      W  7  1          Lille  ...     11.9   
4            Matchweek 4  Sun  Home      D  1  1         Monaco  ...     18.7   

                Expected                           Unnamed: 25_level_0  
    FK PK PKatt       xG npxG npxG/Sh G-xG np:G-xG        Match Report  
0  NaN  1     1      NaN  NaN     NaN  NaN     NaN        Match Report  
1  1.0  0     0      3.5  3.5    0.19  1.5     1.5        Match Report  
2  3.0  1     2      3.2  1.6    0.09  0.8     1.4        Match Report  
3  0.0  0     0      3.4  3.4    0.23  3.6     3.6        Match Report  
4  0.0  1     1      2.7  1.9    0.11 -1.7    -1.9        Match Report  

[5 rows x 26 columns]

In [18]:
shooting.columns = shooting.columns.droplevel()

In [19]:
team_data = matches.merge(shooting[["Date", "Sh", "SoT", "Dist", "FK", "PK", "PKatt"]], on="Date")

In [20]:
team_data.head()

,Date,Time,Comp,Round,Day,Venue,Result,GF,GA,Opponent,...,Formation,Referee,Match Report,Notes,Sh,SoT,Dist,FK,PK,PKatt
0,2022-07-31,21:00,Trophée des Champions,Trophée des Champions,Sun,Home,W,4,0,Nantes,...,3-4-3,Orel Grinfeeld,Match Report,NaN,15,8,NaN,NaN,1,1
1,2022-08-06,21:00,Ligue 1,Matchweek 1,Sat,Away,W,5,0,Clermont Foot,...,3-4-3,Jeremy Stinat,Match Report,NaN,18,12,12.9,1.0,0,0
2,2022-08-13,21:00,Ligue 1,Matchweek 2,Sat,Home,W,5,2,Montpellier,...,3-4-1-2,Willy Delajod,Match Report,NaN,18,8,18.2,3.0,1,2
3,2022-08-21,20:45,Ligue 1,Matchweek 3,Sun,Away,W,7,1,Lille,...,3-4-3,Clément Turpin,Match Report,NaN,16,9,11.9,0.0,0,0
4,2022-08-28,20:45,Ligue 1,Matchweek 4,Sun,Home,D,1,1,Monaco,...,3-4-3,Benoît Bastien,Match Report,NaN,17,4,18.7,0.0,1,1


In [21]:
years = list(range(2023, 2018, -1))
all_matches = []

In [22]:
standings_url = "https://fbref.com/en/comps/13/Ligue-1-Stats"

In [23]:
import time
for year in years:
    data = requests.get(standings_url)
    soup = BeautifulSoup(data.text)
    standings_table = soup.select('table.stats_table')[0]

    links = [l.get("href") for l in standings_table.find_all('a')]
    links = [l for l in links if '/squads/' in l]
    team_urls = [f"https://fbref.com{l}" for l in links]
    
    previous_season = soup.select("a.prev")[0].get("href")
    standings_url = f"https://fbref.com{previous_season}"
    
    for team_url in team_urls:
        team_name = team_url.split("/")[-1].replace("-Stats", "").replace("-", " ")
        data = requests.get(team_url)
        matches = pd.read_html(data.text, match="Scores & Fixtures")[0]
        soup = BeautifulSoup(data.text)
        links = [l.get("href") for l in soup.find_all('a')]
        links = [l for l in links if l and 'all_comps/shooting/' in l]
        data = requests.get(f"https://fbref.com{links[0]}")
        shooting = pd.read_html(data.text, match="Shooting")[0]
        shooting.columns = shooting.columns.droplevel()
        try:
            team_data = matches.merge(shooting[["Date", "Sh", "SoT", "Dist", "FK", "PK", "PKatt"]], on="Date")
        except ValueError:
            continue
        team_data = team_data[team_data["Comp"] == "Ligue 1"]
        
        team_data["Season"] = year
        team_data["Team"] = team_name
        all_matches.append(team_data)
        time.sleep(1)

In [24]:
len(all_matches)

100

In [25]:
match_df = pd.concat(all_matches)

In [26]:
match_df.columns = [c.lower() for c in match_df.columns]

In [27]:
match_df

,date,time,comp,round,day,venue,result,gf,ga,opponent,...,match report,notes,sh,sot,dist,fk,pk,pkatt,season,team
1,2022-08-06,21:00,Ligue 1,Matchweek 1,Sat,Away,W,5,0,Clermont Foot,...,Match Report,NaN,18.0,12.0,12.9,1.0,0.0,0.0,2023,Paris Saint Germain
2,2022-08-13,21:00,Ligue 1,Matchweek 2,Sat,Home,W,5,2,Montpellier,...,Match Report,NaN,18.0,8.0,18.2,3.0,1.0,2.0,2023,Paris Saint Germain
3,2022-08-21,20:45,Ligue 1,Matchweek 3,Sun,Away,W,7,1,Lille,...,Match Report,NaN,16.0,9.0,11.9,0.0,0.0,0.0,2023,Paris Saint Germain
4,2022-08-28,20:45,Ligue 1,Matchweek 4,Sun,Home,D,1,1,Monaco,...,Match Report,NaN,17.0,4.0,18.7,0.0,1.0,1.0,2023,Paris Saint Germain
5,2022-08-31,21:00,Ligue 1,Matchweek 5,Wed,Away,W,3,0,Toulouse,...,Match Report,NaN,20.0,12.0,14.8,2.0,0.0,0.0,2023,Paris Saint Germain
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
41,2019-04-28,15:00,Ligue 1,Matchweek 34,Sun,Away,L,0,3,Nice,...,Match Report,NaN,13.0,3.0,21.4,1.0,0.0,0.0,2019,Guingamp
42,2019-05-04,20:00,Ligue 1,Matchweek 35,Sat,Home,D,0,0,Caen,...,Match Report,NaN,10.0,1.0,20.0,0.0,0.0,0.0,2019,Guingamp
43,2019-05-12,15:00,Ligue 1,Matchweek 36,Sun,Away,D,1,1,Rennes,...,Match Report,NaN,22.0,8.0,17.1,0.0,0.0,1.0,2019,Guingamp
44,2019-05-18,21:00,Ligue 1,Matchweek 37,Sat,Home,D,2,2,Nîmes,...,Match Report,NaN,19.0,9.0,16.2,0.0,0.0,0.0,2019,Guingamp


In [28]:
match_df.to_csv("matches_ligue1.csv")